# How to build the variance quantum circuit

## Prepare environment and  define parameters 

In [ ]:
is_colab = False
import sys
if is_colab:
    !pip install -q torch==1.9.0
    !pip install -q torchvision==0.10.0
    !pip install -q qiskit==0.20.0
    !pip install qfnn
    !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w9VRv0iVfsH20Kb_MkF3yFhFeiYDVy5n' -O model.tar.gz
    !tar zxvf /content/model.tar.gz


import torch
import torchvision

import qiskit
from qiskit import  QuantumCircuit, ClassicalRegister
import numpy as np
import functools

################ Zhirui on 12-30-2020 ################
# path
######################################################
if is_colab:
    data_path = '/content/data' #mnist  path
    resume_path = '/content/model/v16_v2/model_best.tar' #model path
else:
    data_path = '/home/hzr/Software/quantum/qc_mnist/pytorch/data' #mnist  path
    resume_path = '/home/hzr/Software/quantum/QuantumFlow_Tutorial/model/v16_v2/model_best.tar' #model path


In [ ]:
from qfnn.qf_fb.q_output import fire_ibmq,analyze,add_measure
from qfnn.qf_circ.v_lyr_circ import V_LYR_Circ
from qfnn.qf_net.utils import binarize
from qfnn.qf_fb.q_input import UMatrixCircuit
from qfnn.qf_fb.c_input import load_data,to_quantum_matrix
from qfnn.qf_fb.c_qf_mixer import Net
from qfnn.qf_fb.c_input import ToQuantumData
print = functools.partial(print, flush=True)



################ Zhirui on 12-30-2020 ################
# Parameters of settings
######################################################
interest_num = [3,6]
ori_img_size = 28
img_size = 4
num_workers = 0 # number of subprocesses to use for data loading
batch_size = 1# how many samples per batch to load
inference_batch_size = 1
isppd = False #is prepared data



## Load data

In [ ]:
train_loader, test_loader = load_data(interest_num,data_path,isppd,img_size,batch_size,inference_batch_size,False)


for batch_idx, (data, target) in enumerate(test_loader):
    torch.set_printoptions(threshold=sys.maxsize)
    print("Batch Id: {}, Target: {}".format(batch_idx,target))
    quantum_matrix = to_quantum_matrix(data)
    break

## Inference

In [ ]:
################ hzr on 12-30-2020 ################
# Get the parameters of the trained model
######################################################

# Model initialization
checkpoint = torch.load(resume_path, map_location='cpu')

theta1 = checkpoint['state_dict']['fc0.theta']
theta2 = checkpoint['state_dict']['fc1.theta']


### data encoding

In [ ]:
################ hzr on 12-30-2020 ################
# Generate the circuit of u-Matrix
######################################################

#init circuit
circuit = QuantumCircuit()
#define your input and repeat number
u_mat = UMatrixCircuit(4,1)
#add input qubit to your circuit if needed
inputs = u_mat.add_input_qubits(circuit)
#add u-matrix to your circuit
u_mat.forward(circuit,inputs,quantum_matrix)

### build network

In [ ]:
################ hzr on 12-30-2020 ################
# Generate the circuit of v-layer
######################################################

#define your input and repeat number
vqc = V_LYR_Circ(4,1)
#add the first v-layer  to your circuit
vqc.forward(circuit,inputs,'v10',np.array(theta1,dtype=np.double))
#add the second v-layer  to your circuit
vqc.forward(circuit,inputs,'v10',np.array(theta2,dtype=np.double))

circuit.barrier()
#add measurement to your circuit if needed
add_measure(circuit,[inputs[0][3],inputs[0][2]],'reg')

circuit.draw('text',fold=300)

### Simulation

In [ ]:
################ Weiwen on 12-30-2020 ################
# Quantum simulation
######################################################


qc_shots=8192
opt_counts = fire_ibmq(circuit,qc_shots,True)
(opt_mycount,bits) = analyze(opt_counts)
opt_class_prob=[]
for b in range(bits):
    opt_class_prob.append(float(opt_mycount[b])/qc_shots)


print("Simulation Result :",opt_class_prob)
print("Prediction class: {}".format(opt_class_prob.index(max(opt_class_prob))))
print("Target class: {}".format(target[0]))
if opt_class_prob.index(max(opt_class_prob))==target[0]:
    print("Correct prediction")
else:
    print("Incorrect prediction")
print("="*30)

### classical inference

In [ ]:

neural_in_layers = 'v:16,v:2'
layers = []
for item1 in neural_in_layers.split(","):
    x= item1.split(":")
    layer =[]
    layer.append(x[0].strip())
    layer.append(int(x[1].strip()))
    layers.append(layer)

model = Net(img_size,layers,False,False)
model.load_state_dict(checkpoint["state_dict"])

to_quantum_data = ToQuantumData(img_size)
output_data = to_quantum_data(data)
output = model.forward(output_data,False)
print("classical inference:",output)